In [7]:
using SparseArrays
using LinearAlgebra
using Statistics
using SuiteSparse
using Plots
E0 = 1;
Emin = 1e-9;
nu = 0.3;

function top88mma(nelx,nely,volfrac,penal,rmin,ft)
        
    A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12]
    A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6]
    B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4]
    B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2]
    KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11])

    nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx); #number of the nodes in columns
    edofVec = reshape(2*nodenrs[1:end-1,1:end-1].+1,nelx*nely,1) ;#1st dof of each element (x top left)
    edofMat = zeros(nelx*nely, 8); #every line i contains the 8 dof of the ith element
    noeudsvoisins = [0 1 2*nely.+[2 3 0 1] -2 -1];
    for i = 1:8
        for j = 1:nelx*nely
            edofMat[j,i]= edofVec[j]+ noeudsvoisins[i] ;
        end
    end
    #maillage
    iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);# line to build  K
    jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);# columns
    F = spzeros(2*(nely+1)*(nelx+1),1); F[2,1] = -1;
    U = zeros(2*(nely+1)*(nelx+1),1);
    fixeddofs = union(1:2:2*(nely+1),[2*(nelx+1)*(nely+1)]);
    alldofs = 1:2*(nely+1)*(nelx+1);
    freedofs = setdiff(alldofs,fixeddofs);
    #filtering is linear + chain's rule -> matricial opration with operator H
    iH = ones(nelx*nely*(2*(convert(Int64,ceil(rmin))-1)+1)^2);#lines H
    jH = ones(size(iH));# columns
    sH = zeros(size(iH));# values
    k = 0;
    xnew = zeros(nely,nelx)
    for i1 = 1:nelx
        for j1 = 1:nely
            e1 = (i1-1)*nely+j1;
            for i2 = max(i1-(ceil(rmin)-1),1):min(i1+(ceil(rmin)-1),nelx)
                for j2 = max(j1-(ceil(rmin)-1),1):min(j1+(ceil(rmin)-1),nely)
                    e2 = (i2-1)*nely+j2;
                    k = k+1;
                    iH[k] = e1;
                    jH[k] = e2;
                    sH[k] = max(0,rmin-sqrt((i1-i2)^2+(j1-j2)^2));
                end
            end
        end
    end
    H = sparse(iH,jH,sH);# matrix of the Hei
    Hs = [sum(H[i,:]) for i = 1:(size(H)[1])];# sum of the Hei for e fixed -> coeffs used to divide in the filter
    #variables
    x = volfrac*ones(nely,nelx);
    xPhys = x;#directly modified in args of mma
    loop = 0;
    change = 1;
    while change > 0.01
        loop = loop + 1;
        sK = [j*((i+Emin)^penal) for i in ((E0-Emin)*xPhys[:]') for j in KE[:] ];#new values EF
        K = sparse(iK[:],jK[:],sK); K = (K+K')/2;# force to be symmetric
        KK = cholesky(K[freedofs,freedofs]);
        U[freedofs] = KK\(KK'\F[freedofs]);
        
        # analyse, objective function
        edM = [convert(Int64,i) for i in edofMat] ;#to index by int
        mat = (U[edM]*KE).*U[edM];
        ce = reshape([sum(mat[i,:]) for i = 1:(size(mat)[1])],nely,nelx);
        c = sum(sum((Emin*ones(size(xPhys)).+(xPhys.^penal)*(E0-Emin)).*ce));
        dc = -penal*(E0-Emin)*xPhys.^(penal-1).*ce;
        dv = ones(nely,nelx);
        
        #filtering
        if ft == 1
            dc[:] = H*(x[:].*dc[:])./Hs./max(1e-3,maximum(x[:]));
            xnew = mma(x[:],c,dc,dv,nelx,nely,volfrac); 
        elseif ft == 2
            dc[:] = H*(dc[:]./Hs);
            dv[:] = H*(dv[:]./Hs);
            xnew = mma((H*xnew[:])./Hs,c,dc[:],dv[:],nelx,nely,volfrac);
        end
        change = maximum(abs.(x-xnew));
        x = xnew ;
        println("loop = ", loop, "Change = ", change ," c = ", c , " densité struct = ", mean(x) )
        heatmap(x)
        if loop == 50
            return
        end
    end
end

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


top88mma (generic function with 1 method)

In [20]:
#dfdx = dv  = (m x n)-matrix with the derivatives of the constraint functions
 #               f_i with respect to the variables x_j, calculated at xval.
  #              dfdx(i,j) = the derivative of f_i with respect to x_j.

#df0dx = dc = Column vector with the derivatives of the objective function
 #               f_0 with respect to the variables x_j, calculated at xval.
using JuMP # solver
using GLPK
function mma(xval,c,dc,dv,nelx,nely,volfrac)#ici 1 contrainte : (somme(x) <= volfrac) => m =1
    iter = 1; n = nelx*nely ;# pas sur
    toutpetit = 0.001;
    xmax = ones(n) ; xmin =toutpetit*ones(n) ; dx = xmax .- xmin ;
    xmamieps = 0.00001*ones(n,1);
    xmami = max.(dx,xmamieps);
    xmamiinv = ones(n)./xmami;
    xold1 = xval .+1 ;# arbitrary for the moment
    function f(x,low,up) #fonction à optimiser pour MMA
        somme = 0;
        for i = 1:n
            somme += ((x[i] - low[i])^2)/(1-low[i])*dc[i]*x[i];
        end
        return somme# aps sur du 1-low
    end
    while (maximum(abs.(xold1 - xval)) > 0.01) && (iter < 20)
        if iter < 2.5
            low = xval .- 0.5*dx;#Li (xmax = 1, xmin = 0 d'où xmax - xmin = ones(n,1))
            upp = xval .+ 0.5*dx;#Ui ( condition Ui - Li = 1 )
        else
            signes = (xval-xold1).*(xold1-xold2); 
            factor = ones(n);#défnir n qui est la taille de x après
            for i in 1:n
                if signes[i]< 0
                    factor[i] = 0.7;
                elseif signes[i]>0
                    factor[i]= 1.2;
                end
            end
            low = xval - factor.*(xold1 - low);
            upp = xval + factor.*(upp - xold1);
            lowmin = xval .- 10*dx;
            lowmax = xval .- 0.01*dx;
            uppmin = xval .+ 0.01*dx;
            uppmax = xval .+ 10*dx;
            low = max.(low,lowmin);
            low = min.(low,lowmax);
            upp = min.(upp,uppmax);
            upp = max.(upp,uppmin);
        end
"""
        zzz1 = low .+ 0.1*(xval-low);
        zzz2 = xval .- 0.5*dx;
        zzz  = max.(zzz1,zzz2);
        alfa = max.(zzz,xmin);
        zzz1 = upp .- 0.1*(upp-xval);
        zzz2 = xval .+ 0.5*dx;
        zzz  = min.(zzz1,zzz2);
        beta = min.(zzz,xmax);
        
        ux1 = upp.-xval;
        ux2 = ux1.*ux1;
        xl1 = xval.-low;
        xl2 = xl1.*xl1;
        uxinv = ones(n)./ux1;
        xlinv = ones(n)./xl1;
        
        p0 = zeros(n,1);
        q0 = zeros(n,1);
        p0 = max.(dv,0);
        q0 = max.(-dv,0);
        pq0 = 0.001*(p0 .+ q0) .+ toutpetit*xmamiinv;
        p0 = p0 .+ pq0;
        q0 = q0 .+ pq0;
        p0 = p0.*ux2;
        q0 = q0.*xl2;

        P = max.(dc,0);
        Q = max.(-dc,0);
        PQ = toutpetit*((P + Q) .+ xmamiinv');
        P = P .+ PQ;
        Q = Q .+ PQ;
        P = P * sparse(Diagonal(ux2));
        Q = Q * sparse(Diagonal(xl2));println(size(P*uxinv + Q*xlinv))
        #b = P*uxinv .+ Q*xlinv .- fval ;#fval = containtes(xval)
        b = P*uxinv + Q*xlinv .- (sum(xval)-volfrac*n);
        r = -b;
"""
        if iter == 1
            xold1 = xval;
        else
            xold2 = xold1;
            xold1 = xval; 
        end
        iter+=1;
        model = Model(with_optimizer(GLPK.Optimizer))
        #@variable(model, toutpetit <= x[1:n] <= 1);
        println(f(xval,low, upp))
        #@objective(model, Max, f(x,low, upp));
        #@constraint(model, con, sum(x) <= volfrac*n);
        #optimize!(model);
        #c[:] = objective_value(model)
        #xval = value(x);
        #delete(model, x)
    end
    return reshape(xval,nely,nelx);
end

mma (generic function with 1 method)

In [21]:
top88mma(20,10,0.5,3,2.5,2)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = mma(::Array{Float64,1}, ::Float64, ::Array{Float64,1}, ::Array{Float64,1}, ::Int64, ::Int64, ::Float64) at In[20]:94
└ @ Main .\In[20]:94


0.0
loop = 1Change = 0.5 c = 5.719346501537909e11 densité struct = 0.0
0.0
loop = 2Change = 0.0 c = 5.719346501537909e11 densité struct = 0.0


In [22]:
Pkg.add("NLopt")

  Resolving package versions...
  Installed URIParser ──── v0.4.0
  Installed CMakeWrapper ─ v0.2.3
  Installed CMake ──────── v1.2.0
  Installed Parameters ─── v0.12.0
  Installed NLopt ──────── v0.5.1
  Installed BinDeps ────── v1.0.0
  Installed MathProgBase ─ v0.7.8
   Updating `C:\Users\robin\.julia\environments\v1.4\Project.toml`
  [76087f3c] + NLopt v0.5.1
   Updating `C:\Users\robin\.julia\environments\v1.4\Manifest.toml`
  [9e28174c] + BinDeps v1.0.0
  [631607c0] + CMake v1.2.0
  [d5fb7624] + CMakeWrapper v0.2.3
  [fdba3010] + MathProgBase v0.7.8
  [76087f3c] + NLopt v0.5.1
  [d96e819e] + Parameters v0.12.0
  [30578b45] + URIParser v0.4.0
   Building CMake → `C:\Users\robin\.julia\packages\CMake\ULbyn\deps\build.log`
   Building NLopt → `C:\Users\robin\.julia\packages\NLopt\eqN9a\deps\build.log`


In [2]:
#Orginal work Written by Krister Svanberg in Matlab  
#m = nombre de contraintes
xval = x#le x^(0) du pseudo code
epsimin = 10^(-7);
raa0 = 0.00001;
move = 0.5;
albefa = 0.1;
asyinit = 0.5;
asyincr = 1.2;
asydecr = 0.7;
eeen = ones(n,1);
eeem = ones(m,1);
zeron = zeros(n,1);

% Calculation of the asymptotes low and upp :
if iter < 2.5
  low = xval - asyinit*(xmax-xmin);
  upp = xval + asyinit*(xmax-xmin);
else
  zzz = (xval-xold1).*(xold1-xold2);
  factor = eeen;
  factor(find(zzz > 0)) = asyincr;
  factor(find(zzz < 0)) = asydecr;
  low = xval - factor.*(xold1 - low);
  upp = xval + factor.*(upp - xold1);
  lowmin = xval - 10*(xmax-xmin);
  lowmax = xval - 0.01*(xmax-xmin);
  uppmin = xval + 0.01*(xmax-xmin);
  uppmax = xval + 10*(xmax-xmin);
  low = max(low,lowmin);
  low = min(low,lowmax);
  upp = min(upp,uppmax);
  upp = max(upp,uppmin);
end

% Calculation of the bounds alfa and beta :

zzz1 = low + albefa*(xval-low);
zzz2 = xval - move*(xmax-xmin);
zzz  = max(zzz1,zzz2);
alfa = max(zzz,xmin);
zzz1 = upp - albefa*(upp-xval);
zzz2 = xval + move*(xmax-xmin);
zzz  = min(zzz1,zzz2);
beta = min(zzz,xmax);

% Calculations of p0, q0, P, Q and b.

xmami = xmax-xmin;
xmamieps = 0.00001*eeen;
xmami = max(xmami,xmamieps);
xmamiinv = eeen./xmami;
ux1 = upp-xval;
ux2 = ux1.*ux1;
xl1 = xval-low;
xl2 = xl1.*xl1;
uxinv = eeen./ux1;
xlinv = eeen./xl1;
%
p0 = zeron;
q0 = zeron;
p0 = max(df0dx,0);
q0 = max(-df0dx,0);
pq0 = 0.001*(p0 + q0) + raa0*xmamiinv;
p0 = p0 + pq0;
q0 = q0 + pq0;
p0 = p0.*ux2;
q0 = q0.*xl2;
%
P = sparse(m,n);
Q = sparse(m,n);
P = max(dfdx,0);
Q = max(-dfdx,0);
PQ = 0.001*(P + Q) + raa0*eeem*xmamiinv';
P = P + PQ;
Q = Q + PQ;
P = P * spdiags(ux2,0,n,n);
Q = Q * spdiags(xl2,0,n,n);
b = P*uxinv + Q*xlinv - fval ;
%
%%% Solving the subproblem by a primal-dual Newton method # méthode de newton vu qu'on est cvx
[xmma,ymma,zmma,lam,xsi,eta,mu,zet,s] = ...
subsolv(m,n,epsimin,low,upp,alfa,beta,p0,q0,P,Q,a0,a,b,c,d);


UndefVarError: UndefVarError: x not defined